# Import libraries

In [1]:
import pandas as pd
import os
import numpy as np
import catboost
from catboost import *

from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

import xgboost as xgb
import lightgbm as lgb
import time

import warnings
warnings.filterwarnings(action='once')
np.set_printoptions(precision=4)

# import h2o
# from h2o.estimators.gbm import H2OGradientBoostingEstimator
# from h2o.grid.grid_search import H2OGridSearch
# h2o.init(nthreads=-1, strict_version_check=True)

import matplotlib.pyplot as plt

/Users/nazhmeddinbabakhanov/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


# read from dataset 10mlnsourcerowid

In [3]:
train_df = pd.read_csv('10mlnsourcerowid.csv',nrows=100)
train_df.head(5)

,Unnamed: 0,status,sourcerowid,people_id,indexop,indexstart,indexdest,price,mailtype,mailctg,...,start_district_id,start_timezone,dest_area_id,dest_region_id,dest_district_id,op_region_i,op_district_id,op_timezone,total,success_rate
0,967952,1,944502339,1,694500,443096,694500,1998,16,4,...,3.0,4.0,1751.0,8.0,2.0,8.0,2.0,11.0,1,100
1,968003,0,852097698,1,694500,190847,694500,6167,4,4,...,4.0,3.0,1751.0,8.0,2.0,8.0,2.0,11.0,2,50
2,967954,1,694196513,1,694500,111537,694500,4221,4,4,...,9.0,3.0,1751.0,8.0,2.0,8.0,2.0,11.0,3,66
3,967992,1,824189587,1,694500,170972,694500,11069,4,4,...,9.0,3.0,1751.0,8.0,2.0,8.0,2.0,11.0,4,75
4,967993,1,825071827,1,694500,170972,694500,11069,4,4,...,9.0,3.0,1751.0,8.0,2.0,8.0,2.0,11.0,5,80


In [4]:
train_df.T

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Unnamed: 0,9.679520e+05,9.680030e+05,9.679540e+05,9.679920e+05,9.679930e+05,9.679950e+05,9.679960e+05,9.679940e+05,9.679490e+05,9.679970e+05,...,9.676520e+05,9.676540e+05,9.676530e+05,9.676550e+05,9.676560e+05,9.676600e+05,9.676440e+05,9.676580e+05,9.676590e+05,9.676430e+05
status,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
sourcerowid,9.445023e+08,8.520977e+08,6.941965e+08,8.241896e+08,8.250718e+08,8.273817e+08,8.277936e+08,8.272031e+08,7.238613e+08,8.307382e+08,...,8.349555e+08,8.389817e+08,8.373094e+08,8.393234e+08,8.421391e+08,8.554640e+08,2.625564e+08,8.452385e+08,8.453562e+08,2.619141e+08
people_id,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
indexop,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,...,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05
indexstart,4.430960e+05,1.908470e+05,1.115370e+05,1.709720e+05,1.709720e+05,1.709710e+05,1.709710e+05,1.709710e+05,1.275760e+05,1.709710e+05,...,1.709710e+05,1.709710e+05,1.709710e+05,1.709710e+05,1.709710e+05,1.709720e+05,1.709710e+05,1.709720e+05,1.709710e+05,1.709710e+05
indexdest,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,6.945000e+05,...,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05,6.830090e+05
price,1.998000e+03,6.167000e+03,4.221000e+03,1.106900e+04,1.106900e+04,2.199000e+03,1.190000e+03,1.790000e+03,1.548000e+03,3.270000e+03,...,5.183000e+03,4.463000e+03,3.209000e+03,9.659000e+03,5.009000e+03,6.517000e+03,2.799000e+03,6.918000e+03,2.699000e+03,5.668000e+03
mailtype,1.600000e+01,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,1.600000e+01,4.000000e+00,...,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
mailctg,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,...,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00


In [4]:
train_df = train_df[train_df.sourcerowid > 1315349602]

# deleting columns  

In [5]:
train_df.drop(["Unnamed: 0","decisiontime","optimestamp","sourcerowid"], axis=1, inplace=True) #"Unnamed: 0","decisiontime"


In [6]:
train_df.to_csv("test.csv")

# info about columns

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9527004 entries, 0 to 9527003
Data columns (total 29 columns):
Unnamed: 0           int64
status               int64
sourcerowid          int64
people_id            int64
indexop              int64
indexstart           int64
indexdest            int64
price                int64
mailtype             int64
mailctg              int64
mailmass             int64
optimestamp          int64
decisiontime         float64
mailrank             int64
sendctgid            int64
postmark             int64
gender               int64
start_area_id        float64
start_region_id      float64
start_district_id    float64
start_timezone       float64
dest_area_id         float64
dest_region_id       float64
dest_district_id     float64
op_region_i          float64
op_district_id       float64
op_timezone          float64
total                int64
success_rate         int64
dtypes: float64(11), int64(18)
memory usage: 2.1 GB


In [6]:
# training_df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

# division by X and y 

In [7]:
y = train_df.status
X = train_df.drop('status', axis=1)
cat_features = [1, 2, 5, 6, 7, 8, 10, 11]
neg_count = len(y) - sum(y)
pos_count = sum(y)
scale_pos_weight = neg_count / pos_count


print('Labels: {}'.format(set(y)))
print('Negative count = {}, Positive count = {}'.format(neg_count, pos_count))
print(scale_pos_weight)

Labels: {0, 1}
Negative count = 1380813, Positive count = 6989685
0.19755010418924457


# train_test_split

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size = 0.1, random_state=123)

#  RandomizedSearchCV with roc_auc

In [9]:
def randomized_cv(model, param_grid, x_train=x_train, y_train=y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='roc_auc')
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    print(grid_search.best_score_)
    return grid_search.best_estimator_

# Checking model

In [ ]:
def apply_model(model,test_x=x_test, test_y=y_test):
    pred_y_classifier = model.predict(test_x)
    y_score = model.predict_proba(test_x)
    accuracy_score = metrics.accuracy_score(test_y, pred_y_classifier)
    precision_score = metrics.precision_score(test_y, pred_y_classifier)
    recall_score = metrics.recall_score(test_y, pred_y_classifier)
    f1_score = metrics.f1_score(test_y, pred_y_classifier)
    auc = metrics.roc_auc_score(test_y, pred_y_classifier)
    roc_curve = metrics.roc_curve(test_y, y_score[ :, 1 ])
    print('Auc_score: %f '%auc)
    print('f1_score: %f1'%f1_score)
    print('Acc: %f3'%accuracy_score)
    return auc

#  Saving in csv

In [ ]:
def to_csv(name, model, auc_lgb):
    print("Test file")
    sub = pd.DataFrame({"real":y_test, "predic":model.predict(x_test), "predict_proba_0":model.predict_proba(x_test)[:,0], "predict_proba_1":model.predict_proba(x_test)[:,1] })
    y_pred = model.predict(x_test)
    sub.to_csv(name+" "+str(auc_lgb)+".csv", index=False) 

# model in LGBMClassifier

In [ ]:
param_grid = {
    'max_depth': [2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.1, 0.3, 0.01, 0.02, 0.05]
}
model = randomized_cv(lgb.LGBMClassifier(), param_grid)
auc_lgb = apply_model(model, x_test , y_test)
to_csv("LGBMClassifier", model, auc_lgb)

/Users/nazhmeddinbabakhanov/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
auc_lgb

# model in XGBClassifier

In [19]:
import xgboost as xgb

param_grid = {
    'max_depth': [2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.1, 0.3, 0.01, 0.02, 0.05]
}
xgb = randomized_cv(xgb.XGBClassifier(), param_grid)
auc_xgb = apply_model(xgb)

model XGBClassifier best accuracy score is 0.9206403892349059
time for training is 8523.760426998138 seconds
0.9206403892349059
Auc_score: 0.747836 
f1_score: 0.9186881
Acc: 0.8671003


# model in CatBoostClassifier

In [20]:
param_grid = {
    'iterations': [2, 3, 4, 5, 6],
    'depth': [2, 3, 4, 5, 6],
    'learning_rate': [0.1, 0.3, 0.01, 0.02, 0.05]
}
cbm = randomized_cv(CatBoostClassifier(), param_grid)
print("Test")
auc_cbm = apply_model(cbm)
to_csv("CatBoostClassifier", model, auc_cbm)

0:	learn: 0.6224127	total: 697ms	remaining: 2.79s
1:	learn: 0.5769105	total: 1.25s	remaining: 1.88s
2:	learn: 0.5409759	total: 1.83s	remaining: 1.22s
3:	learn: 0.5084764	total: 2.43s	remaining: 608ms
4:	learn: 0.4796498	total: 3.02s	remaining: 0us
model CatBoostClassifier best accuracy score is 0.9066105776280229
time for training is 678.8024830818176 seconds
0.9066105776280229
Test
Auc_score: 0.731319 
f1_score: 0.9140831
Acc: 0.8590903
Test file


In [ ]:
# model = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8)
# model = GradientBoostingClassifier()

model = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=7,max_features='sqrt',subsample=0.8)
model.fit(X, y)

clfs = []
for i in range(24):
    clf = GradientBoostingClassifier(learning_rate=0.03,random_state=100 + i, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8)
    clf2 = xgb.XGBClassifier(max_depth=7, learning_rate=0.06, n_estimators=180,scale_pos_weight=5, silent=True, objective='binary:logistic')
    
    clf = lgb.LGBMClassifier(
                         objective='multiclass',
                         boosting_type='dart',
                         colsample_bytree=0.1, #训练特征采样率 列
                         learning_rate=0.02,
                         min_child_samples=19,
                         num_leaves=33,
                         subsample=0.552,
                         n_jobs=-1,
                         n_estimators=700,
                         silent=True,
                         scale_pos_weight=1,
                         metric='None',
                         random_state=300 + i,
                         class_weight='balanced'
    )
clfs.append(('lgbm{}'.format(i), clf))
# clfs.append(('clf1{}'.format(i), clf1))
# clfs.append(('clf2{}'.format(i), clf2))

model = VotingClassifier(clfs, voting='soft')

model.fit(data_x, data_y)
# model.fit(X_res, y_res)


# res_y = model.predict(X_res)
res_y = model.predict(data_x)

scores = accuracy_score(res_y, y_res)